In [3]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.formula.api as sm
from statsmodels import regression
import statsmodels

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc, rcParams

# Chap 14. 분산분석
## 14.1 서론
- 셋 이상의 모집단 간의 평균을 비교하는 방법
- 관측값들이 달라지는 것을 여러 요인으로 나누어 각 요인들이 얼마나 변화의 정도에 기여하였는가 분석

## 14.2 일원배치 분산분석
- 관측값 분해 : 개개 관측값의 총평균에 대한 편차 $(y_{ij}-\bar{y})$, 각 코팅처리 간의 평균값의 차이에 기인하는 부분 $(\bar{y_i})$ 동일한 코팅처리 내에서 발생하는 측정값의 오차에 의한 부분, $(y_{ij}-\bar{y_i})$ $$관측값\ = \ (총평균)\ +\ (처리에\ 의한\ 편차)\ + \ (잔차) $$ $$ y_{ij}\ =\quad \bar{y}\quad (\bar{y_i}-\bar{y})\quad +\ (y_{ij}-\bar{y_i})$$
- 만일 네 코팅에서 얻어진 평균 마모도 차이가 없다면 $(\bar{y_i}-\bar{y})$는 0에 가까워야 함
  - 전체 처리효과들의 변동을 측정하는 양으로, 이 행렬의 모든 원소 제곱합을 하면 **처리제곱합**(treatment sum of squares, SStr) $$SStr = \sum^n_{i=1} n_i(\bar{y_i}-\bar{y})^2$$
- $(y_{ij}-\bar{y_i})$ : 각 관측값과, 관측값이 속한 처리 평균과의 편차 => 실험에 사용되는 물질 자체의 변동값이나, 측정기기의 변동값 반영 / **랜덤오차**
  - 모든 랜덤오차의 제곱합 : **오차제곱합**(error sum of squares, SSE), 각 처리마다 잔차의 제곱합을 구한 후, 모든 처리에서 그 값을 합한 것 $$SSE = \sum^n_{i=1} \sum^{n_i}_{j=1}(y_ij-\bar{y_i})^2$$
- $(y_{ij}-\bar{y})$ : 개개 관측치의 총평균에 대한 잔차, 모든 편차들의 제곱합, **총제곱합**(Total sum of squares, SST) $$SST = \sum^n_{i=1} \sum^{n_i}_{j=1}(y_{ij}-\bar{y})^2$$
> $$SST\ =\ SStr\ +\ SSE$$
>
> (제곱합의 자유도) = (제곱을 하여 더하는 항의 수) - (각 항들에 의하여 만족되는 선형 제약조건의 수)

예시)
- 처리제곱합은 4개 항의 합 $\sum^4_{i=1}n_i(\bar{y_i}-\bar{y})^2$
- 각 항들은 다음의 한 가지 제약조건 만족 $\sum^n_{i-1}n_i (\bar{y_i}-\bar{y})=0$
- 총평균 $\bar{y}$는 처리평균들의 가중평균의 제약조건 등호 성립 $\bar{y}=\frac{\sum n_i\bar{y_i}}{\sum n_i}$
- 결과적으로 처리제곱합 자유도 : 4개 case -1 / 오차제곱합의 자유도 : 총 데이터갯수 - 4개 case / 총제곱합 자유도 총 데이터 갯수 - 1
$$\sum^k_{i=1}\sum^{n_i}_{j=1}(y_{ij}-\bar{y})^2 = \sum^k_{i=1}n_i(\bar{y_i}-\bar{y})^2+\sum^k_{i=1}\sum^{n_i}_{j=1}(y_{ij}-\bar{y_i})^2$$
$$총제곱합(\sum^k_{i=1}n_i-1)\ =\ 처리제곱합(k-1)\ +\ 오차제곱합(\sum^k_{i=1}n_i-k)$$
- 평균제곱(mean square) = $\frac{제곱합}{자유도}$

### 제곱합의 간편 계산식
$T_i=\sum^{n_i}_{j=1}y_{ij}$ : 처리 i에서의 모든 관측값의 합계

$T=\sum^k_{i=1}T_i=\sum^{k}_{i=1}\sum^{n_i}_{j=1}y_{ij}$ : 모든 관측값의 총계
$$SST=\sum^k_{i=1}\sum^{n_i}_{j=1}y_{ij}^2-\frac{T^2}{n},$$

In [4]:
# example 4
y = np.array([10,15,8,12,15,14,18,21,15,17,16,14,15,17,15,18,12,15,17,15,16,15])
treat = np.repeat(['A','B','C','D'], [5,4,7,6])
data = pd.DataFrame({'y':y, 'treat':treat})
data.head()

,y,treat
0,10,A
1,15,A
2,8,A
3,12,A
4,15,A


In [5]:
from statsmodels.stats.anova import anova_lm
import statsmodels.formula.api as sm

lmfit = sm.ols('y ~ treat', data=data).fit()
anova_lm(lmfit)

,df,sum_sq,mean_sq,F,PR(>F)
treat,3.0,68.0,22.666667,4.340426,0.018136
Residual,18.0,94.0,5.222222,NaN,NaN


In [8]:
# example 1
a = [10,15,8,12,15] ; b = [14,18,21,15] ; c = [17,16,14,15,17,15,18] ; d = [12,15,17,15,16,15]

In [9]:
# example 2
ta = np.sum(a) ; tb = np.sum(b) ; tc = np.sum(c) ; td = np.sum(d)
t = ta + tb + tc + td
t2 = np.sum(np.power(a,2)) + np.sum(np.power(b,2)) + np.sum(np.power(c,2)) + np.sum(np.power(d,2))

sst = t2 - t**2/(len(a) + len(b) + len(c) + len(d))
sstr = ta**2/len(a) + tb**2/len(b) + tc**2/len(c) + td**2/len(d) - t**2/(len(a) + len(b) + len(c) + len(d))
sse = sst - sstr
print(sst, sstr, sse)

162.0 68.0 94.0


In [10]:
# prob 2.2
a = [35,24,28,21] ; b = [19,14,14,13] ; c = [21,16,21,14]
ta = np.sum(a) ; tb = np.sum(b) ; tc = np.sum(c)
t = ta + tb + tc
t2 = np.sum(np.power(a,2)) + np.sum(np.power(b,2)) + np.sum(np.power(c,2))

sst = t2 - t**2/(len(a) + len(b) + len(c))
sstr = ta**2/len(a) + tb**2/len(b) + tc**2/len(c) - t**2/(len(a) + len(b) + len(c) )
sse = sst - sstr

k = 3
df_tr = k - 1 ; df_e = (len(a) + len(b) + len(c)) - k ; df = df_tr + df_e

mstr = sstr / (k-1)
mse = sse / (len(a)+len(b)+len(c)-k)

print(f"처리 : {sstr:.4f}, {df_tr}, {mstr}")
print(f"오차 : {sse:.4f}, {df_e}, {mse}")
print(f"합계 : {sst:.4f}, {df}")

처리 : 312.0000, 2, 156.0
오차 : 170.0000, 9, 18.88888888888889
합계 : 482.0000, 11


In [11]:
# prob 2.4

## 14.3 일원배치 분산분석모형에서의 추론

### F분포
- k개 모집단 모평균 차이가 없다는 귀무가설이 있을 때, 귀무가설이 맞다면 $\bar{y_i}-\bar{y}$ 값들이 작아질 것이고, 이것의 함수인 평균처리 제곱($MStr=\sum n_i(\bar{y_i}-\bar{y})^2$도 작아질 것. 귀무가설이 기각되면 평균처리 제곱도 커질 것
- 평균처리제곱의 크기에 따라 귀무가설 기각 여부를 결정하는데, 그 기준으로 공통분산 추정치인 평균오차제곱($MSE=s^2$)이 쓰일 것
=> 평균오차제곱에 대한 평균처리제곱의 비율($\frac{MStr}{MSE}$)에 따라 기각여부 결정. 이 통계량은 모평균이 동일하다는 귀무가설 하에서 자유도가 (k-1, n-k)인 F분포를 따른다. $$F=\frac{MStr}{MSE}=\frac{SStr/(k-1)}{SSE/(n-k)}\sim F(k-1, n-k)$$
- 분자와 분모의 자유도에 의하여 결정 : $F_{\alpha}(v_i, v_2), \quad F_{1-\alpha}(v_1, v_2)=\frac{1}{F_\alpha(v_2, v_1)}$
> **F분포를 이용한 모평균의 동일성 검정**
>
>$$F=\frac{MStr}{MSE}=\frac{SStr/(k-1)}{SSE/(n-k)}\sim F(k-1, n-k)$$
>검정통계량의 분포는 귀무가설이 맞을 때 자유도가 (k-1, n-k)인 F분포를 따르고 유의수준 $\alpha$의 기각역은 다음과 같음 $$R:F=\frac{MStr}{MSE}\geq f_\alpha(k-1, n-k)$$
> 이때 $n=\sum^k_{i=1}n_i$, $F_\alpha(k-1, n-k)$는 자유도 (k-1, n-k)인 F분포에서의 상위 $\alpha$확률을 주는 경계값

In [2]:
# example 3
a = [10,15,8,12,15] ; 
alpha = 0.05